# Anomaly Detection v2.1

## Note:
It is recommended to run the code using a GPU. To do this, go to Runtime > Change runtime type > Hardware Accelerator > select GPU.

It may be the case that a GPU is not available, in which case, use a default CPU ("None" hardware accelerator). The code will still work without a GPU, but may run much slower.

### Contributers:
Jason Ding

## Pre-requisites

In the following cells, we are installing and importing the necessary libaries and downloading the classification model.

In [1]:
!wget https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
!wget https://raw.githubusercontent.com/hendrycks/pre-training/master/robustness/adversarial/models/wrn_with_pen.py
!pip3 install torchvision==0.12.0

--2022-08-16 14:31:13--  https://github.com/hendrycks/outlier-exposure/raw/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt [following]
--2022-08-16 14:31:14--  https://raw.githubusercontent.com/hendrycks/outlier-exposure/master/CIFAR/snapshots/baseline/cifar10_wrn_baseline_epoch_99.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9037421 (8.6M) [application/octet-stream]
Saving to: ‘cifar10_wrn_baseline_epoch_99.pt’

cifar10_wrn_baselin 

In [2]:
import torch
import math
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
import numpy as np
import torch.nn.functional as F
import sklearn.metrics as sk
from wrn_with_pen import WideResNet

prefetch = 2

## Model and Data Loading

In the following cells, we are loading the CIFAR-10 model. Additionally, we are retrieving CIFAR-10 and out-of-distribution datasets.

In [3]:
# Load CIFAR-10 model

net = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)

if torch.cuda.is_available():
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt'))
    net.eval()
    net.cuda()
else:
    net.load_state_dict(torch.load('cifar10_wrn_baseline_epoch_99.pt', map_location=torch.device('cpu')))
    net.eval()

In [4]:
# /////////////// Loading Datasets ///////////////
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

# /////////////// CIFAR-10 ///////////////

data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

cifar_10_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar10_data = cifar_10_data
cifar10_loader = torch.utils.data.DataLoader(cifar10_data, batch_size=200, shuffle=False,
                                          num_workers=prefetch, pin_memory=True)
ood_num_examples = len(cifar10_data) // 5

# /////////////// Rademacher Noise ///////////////

dummy_targets = torch.ones(ood_num_examples)
ood_data = torch.from_numpy(np.random.binomial(
    n=1, p=0.5, size=(ood_num_examples, 3, 32, 32)).astype(np.float32)) * 2 - 1
rademacher_ood_data = torch.utils.data.TensorDataset(ood_data, dummy_targets)
rademacher_ood_loader = torch.utils.data.DataLoader(rademacher_ood_data, batch_size=200, shuffle=True)

# /////////////// SVHN ///////////////

data_transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize(mean, std)])

svhn_ood_data = torchvision.datasets.SVHN(root = "data", 
                          split="test",
                          transform = data_transform, 
                          download = True)

svhn_ood_loader = torch.utils.data.DataLoader(svhn_ood_data, batch_size=200, shuffle=True,
                                         num_workers=prefetch, pin_memory=True)

# /////////////// DTD ///////////////

# data_transform = transforms.Compose([transforms.Resize(32), transforms.CenterCrop(32),
#                                      transforms.ToTensor(), transforms.Normalize(mean, std)])

# dtd_ood_data = torchvision.datasets.DTD(root = "data", 
#                           split="test",
#                           transform = data_transform, 
#                           download = True)

# dtd_ood_loader = torch.utils.data.DataLoader(dtd_ood_data, batch_size=200, shuffle=True,
#                                          num_workers=prefetch, pin_memory=True)

# /////////////// CIFAR-100 ///////////////

data_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])

cifar100_ood_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=data_transform
)

cifar100_ood_loader = torch.utils.data.DataLoader(cifar100_ood_data, batch_size=200, shuffle=True,
                                         num_workers=prefetch, pin_memory=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data


  0%|          | 0/64275384 [00:00<?, ?it/s]

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


## General Functions

The following cells define functions that we will use in the rest of the code. You do not need to do anything here.

In [5]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.cpu().numpy()

'''
Calculates the anomaly scores for a portion of the given dataset. 
If a GPU is not available, will run on a smaller fraction of the
dataset, so that calculations will be faster.

loader: A DataLoader that contains the loaded data of a dataset
anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
model_net: The classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def get_ood_scores(loader, anomaly_score_calculator, model_net, use_penultimate = False):
    _score = []

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loader):
            if torch.cuda.is_available():
                fraction = 200
            else:
                fraction = 1000
            if batch_idx >= ood_num_examples // fraction:
                break

            if torch.cuda.is_available():
                data = data.cuda()

            output = model_net(data)

            if use_penultimate:
                score = anomaly_score_calculator(output[0], output[1])
            else:
                score = anomaly_score_calculator(output[0])
            _score.append(score)

    return concat(_score).copy()

In [6]:
# /////////////// Printing Results ///////////////
all_anomaly_results = {}

'''
Returns and prints out the AUROC score of a dataset.

ood_loader: A DataLoader that contains the loaded data of a dataset
anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
model_net: The classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def get_and_print_results(ood_loader, anomaly_score_calculator, model_net, use_penultimate):
    out_score = get_ood_scores(ood_loader, anomaly_score_calculator, model_net, use_penultimate = use_penultimate)
    auroc = get_auroc(out_score, in_score)
    print('AUROC: \t\t\t{:.2f}'.format(100 * auroc) + "%")
    return auroc

'''
Prints out the AUROC score of all the OOD datasets. The results 
will be appended to global variable all_anomaly_results, which 
is used later for display purposes.

anomaly_score_calculator: A function that takes in the output 
                          logit of a batch of data and/or the 
                          penultimate.
anomaly_score_name: The name of the anomaly score method.
model_net: The classifier model.
model_name: The name of the classifier model.
usePenultimate: True if anomaly_score_calculator needs the 
                penultimate as a parameter. False otherwise.
'''
def print_all_results(anomaly_score_calculator, anomaly_score_name, model_net, model_name = "default_model", use_penultimate = False):
    global in_score, all_anomaly_results
    in_score = get_ood_scores(cifar10_loader, anomaly_score_calculator, model_net, use_penultimate)
    results = []

    print('Rademacher Noise Detection')
    auroc = get_and_print_results(rademacher_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    print('\nSVHN Detection')
    auroc = get_and_print_results(svhn_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    # print('\nDTD Detection')
    # auroc = get_and_print_results(dtd_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    # results.append(auroc)

    print('\nCIFAR-100 Detection')
    auroc = get_and_print_results(cifar100_ood_loader, anomaly_score_calculator, model_net, use_penultimate)
    results.append(auroc)

    average = sum(results) / len(results)
    results.append(average)

    if not model_name in all_anomaly_results:
        all_anomaly_results[model_name] = {}
    all_anomaly_results[model_name][anomaly_score_name] = results

## Implement AUROC Score
Fill in the get_auroc score. We will use this function in order to calculate the AUROC score of an out-of-distribution dataset.

It may be helpful to use the sklearn.metrics.roc_auc_score() function. Both _pos and _neg should be used.

In [7]:
'''
Calculates the AUROC score of a OOD dataset.

_pos: an array of anomoly scores of the OOD dataset
_neg: an array of anomoly scores of images in the CIFAR-10 dataset
return: The AUROC score the data in decimal form
'''
def get_auroc(_pos, _neg):
    ############################################################################
    # TODO:  Calculate the AUROC score.                                        #
    ############################################################################
    scores = torch.from_numpy(np.concatenate((_pos, _neg)))
    if any(e < 0 or 1 < e for e in _pos + _neg):
      scores = torch.nn.functional.softmax(scores)
    pn = len(_pos)
    nn = len(_neg)
    y_true = np.repeat(0, pn + nn)
    y_true[:pn] = 1
    auroc_score = sk.roc_auc_score(y_true, scores)
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return auroc_score

## Implement Anomaly Score Calculators
Fill in the folllowing functions which calculate the anomaly score given the model's output for a batch of data (the output will contain one logit per image in the batch).

The following equations show how the logits should be transformed in order to get the anomaly score.

Max Logit Equation: <br>
$\text{Score}=-\text{max} l_k$

Max Softmax Equation: <br>
$\text{Score}=-\text{max} p(y=k|x)$

Cross Entropy Anomaly Equation: <br>
$\text{Score} = \bar{l}-\text{log}∑_{c=1}^{\text{num_classes}}e^{l_c}$

In [30]:
# /////////////// Anomaly Score Calculators ///////////////

'''
Calculates the max logit anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def max_logit_anomaly_score(output):
    ############################################################################
    # TODO: Calculate the max logit anomaly score (returning a numpy array).   #
    ############################################################################
    score = np.negative(to_np(output).max(axis=1))
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score

'''
Calculates the max softmax anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def max_softmax_anomaly_score(output):
    ############################################################################
    # TODO: Calculate the max softmax anomaly score (returning a numpy array). #
    ############################################################################
    score = np.negative(to_np(output.softmax(axis=1)).max(axis=1))
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return score

'''
Calculates the cross entropy anomaly score of a batch of outputs.

output: The model's output for a batch of data.
'''
def cross_entropy_anomaly_score(output):
    ############################################################################
    # TODO: Calculate the cross entropy anomaly score (returning a numpy array).#
    ############################################################################
    logits = torch.tensor(output)
    right = logits.exp().sum(dim=1).log()
    left = logits.max(dim=1).values
    scores = right - left
    scores = to_np(scores)

    #sd = torch.tensor(output).softmax(axis=1)  # Softmax distribution
    #_, d = sd.shape
    #scores = -((-sd.log().sum(axis=1)) / d)
    #scores = to_np(scores)
    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    return scores

## Print AUROC Results

Run the following cells in order to see how well each of the anomaly score calculators do on the OOD datasets.

In [27]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", net)

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
AUROC: 			69.77%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			91.43%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			86.99%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [28]:
print("======= Max Softmax AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax", net)

======= Max Softmax AUROC Scores =======
Rademacher Noise Detection
AUROC: 			80.10%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			92.32%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			88.72%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [31]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", net)

======= Cross Entropy AUROC Scores =======


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Rademacher Noise Detection
AUROC: 			80.10%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


AUROC: 			91.99%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


AUROC: 			88.40%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


## Implement ViM

We will now implement virtual-logit matching (ViM). ViM works by doing the following. 

ViM calculates the **principal space** ($P^\perp$) using the training data's (CIFAR-10)  feature (penultimate). Let us use the 12 most significant principal components for our principal space. HINT: You may find it helpful to use np.linalg.svd and then principal component analysis. One can also use covarience to find the principal space.

Then, ViM calculates **alpha** ($\alpha$) by projecting the training data's features onto the prinicpal space calculated before. This is known as the **residual** ($x^{P^\perp}$). Alpha is then calculated by the following equation:
> $\alpha := \frac{∑^k_{i=1}\text{max}_{j=1,...,C}\{l^i_j\}}{∑^K_{i=1}\| x_i^{P\perp} \|}$

In other words, alpha is the sum of each logit's maximum value, divided by the sum of the norms of each feature's residuals.

Your calculated alpha in this part should be in the range of 1.5 to 1.9.


In [43]:
# You may find the following functions useful.
from numpy.linalg import pinv, norm
from scipy.special import logsumexp

_score = []
to_np = lambda x: x.data.cpu().numpy()
concat = lambda x: np.concatenate(x, axis=0)

# Extraction fully connected layer's weights and biases
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
# Origin of a new coordinate system of feature space to remove bias
u = -np.matmul(pinv(w), b)

'''
Calculates and returns the principal space and alpha values given
the training data the model used.

training_data_loader: A DataLoader that contains the loaded data of a 
                      training dataset.
model_net: The classifier model.
verbose: If true, will print out the alpha value.
return: Returns a tuple whose first element is the principal space matrix
        and second element is the alpha value.
'''
def compute_ViM_principal_space_and_alpha(training_data_loader, model_net, verbose = False):
    result = []

    # Getting the first batch of the training data to calculate principal space and alpha
    training_data, target = next(iter(training_data_loader))
    if torch.cuda.is_available():
        training_data = training_data.cuda()

    result = model_net(training_data)
    logit = result[0] # Logits (values before softmax)
    penultimate = result[1] # Features/Penultimate (values before fully connected layer)

    logit_id_train = logit.cpu().detach().numpy().squeeze()
    feature_id_train = penultimate.cpu().detach().numpy().squeeze()

    ############################################################################
    # TODO:  Calculate the pricipal space and then comput alpha.               #
    ############################################################################

    if verbose:
        print('Computing principal space...')
    m, D = feature_id_train.shape
    x = feature_id_train - u # (200, 128)

    # Covariance matrix:
    Sigma = (1/m) * x.T @ x  # (128,200) @ (200, 128) = (128, 128)
    print('x.shape', x.shape)
    U, _, _ = np.linalg.svd(Sigma) # U (128, 128)
    k = 12  # Let us use the 12 most significant principal components for our principal space.
    principal_space = U[:, k:] 

    if verbose:
        print('Computing alpha...')

    xp = x @ principal_space  # = (200, * )
    xp_n_s = norm(xp, axis=-1).sum()
    alpha = logit.max(dim=1).values.sum() / xp_n_s

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    if verbose:
        print(f'alpha = {alpha}')

    return principal_space, alpha

principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, net, verbose = True)

Computing principal space...
x.shape (200, 128)
Computing alpha...
alpha = 16.160999298095703


## Implement ViM Anomaly Score Calculator

Now, implement the ViM anomaly score calculator. 

First, we want to project our penultimate/feature values onto the principal space which we found before, which is called the residual. We multiply the norm of this residual by alpha to get what we call the **virtual logit score**. 

Next, we get what we call the **energy score** by taking LogSumExp of the logits. 

Finally, the outputted **anomaly score** is calculated by subtracting the virtual logit by the energy score.

$\text{vlogit}= \alpha \| {x^{P^\perp}} \|$\
$\text{energy} = \text{ln}\sum_{i=1}^C e^{l_i}$\
$\text{anomaly_score} = \text{vlogit} - \text{energy}$

In [44]:
'''
Calculates the ViM anomaly score of a batch of outputs.

output: The model's output for a batch of data.
penultimate: The model's penultimate (feature values) for a batch of data.
'''
def ViM_anomaly_score_calculator(output, penultimate):
    logit_id_val = output.cpu().detach().numpy().squeeze()
    feature_id_val = penultimate.cpu().detach().numpy().squeeze()

    ############################################################################
    # TODO:  Calculate the anomaly score.                                      #
    ############################################################################
    ps = principal_space
    x = feature_id_val - u # (m, 128)
    xp = x @ ps  # = (m, 12)
    xp_n = norm(xp, axis=-1)
    vl = (alpha.cpu() * xp_n)
    energy = np.log(np.exp(logit_id_val).sum())
    score_id = to_np(vl) - energy

    ############################################################################
    #                             END OF YOUR CODE                             #
    ############################################################################
    
    return score_id

print("======= ViM_anomaly_score_calculator =======")
w, b = net.fc.weight.cpu().detach().numpy(), net.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, net) # Making sure you have the correct ViM values before calculating the score 
print_all_results(ViM_anomaly_score_calculator, "ViM", net, use_penultimate = True)

======= ViM_anomaly_score_calculator =======
x.shape (200, 128)
Rademacher Noise Detection
AUROC: 			99.94%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			95.20%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			82.82%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


## Compare Anomaly Score Results
Run the following cell to see how the different anomaly score calculators compare to each other for the OOD datasets. You should see that ViM is superior to other anomaly scores in all of the datasets.

In [45]:
# ///////////////// Compare Results /////////////////

def get_results_max(model_name = "normal"):
    all_anomaly_results[model_name]["max"] = [0,0,0,0,0]
    for key in all_anomaly_results[model_name].keys():
        if (key != "max"):
            index = 0
            for score in all_anomaly_results[model_name][key]:
                all_anomaly_results[model_name]["max"][index] = \
                    max(score, all_anomaly_results[model_name]["max"][index])
                index += 1


def compare_all_results():
    for model_name in all_anomaly_results:
        to_be_printed = " " * (25 - len(model_name)) + model_name
        dataset_names = ["Rademacher", "SVHN", "CIFAR-100", "Average"]
        for name in dataset_names:
            to_be_printed += " | " + " "*(6-math.ceil(len(name)/2)) + \
                                name + " "*(6-math.floor(len(name)/2))

        print(to_be_printed)
        print("=" * (25 + len(dataset_names) * 15))

        get_results_max(model_name = model_name)
        for key in all_anomaly_results[model_name].keys():
            if (key != "max"):
                to_be_printed = " "*(25-len(key)) + key
                index = 0
                for result in all_anomaly_results[model_name][key]:
                    if (all_anomaly_results[model_name]["max"][index] == result):
                        result = "*" + '{:.2f}'.format(round(result * 100, 2)) + "%"
                    else:
                        result = '{:.2f}'.format(round(result * 100, 2)) + "%"
                    to_be_printed += " | " + " "*(6-math.ceil(len(result)/2)) + \
                                        result + " "*(6-math.floor(len(result)/2))
                    index += 1
                print(to_be_printed)
        print()

    print("\n* highlights the maximum AUROC Score for an OOD Dataset")

compare_all_results()

            default_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    69.77%    |    91.43%    |    86.99%    |    82.73%   
              Max Softmax |    80.10%    |    92.32%    |   *88.72%    |    87.05%   
            Cross Entropy |    80.10%    |    91.99%    |    88.40%    |    86.83%   
                      ViM |   *99.94%    |   *95.20%    |    82.82%    |   *92.65%   

     pixmix_trained_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |   *94.08%    |    92.25%    |    87.42%    |    91.25%   
  Max Softmax Probability |    92.76%    |    91.23%    |    87.92%    |    90.64%   
            Cross Entropy |    92.44%    |    91.24%    |    87.72%    |    90.46%   
                      ViM |    93.88%    |   *94.78%    |   *91.47%    |   *93.37%   


* highlights the maximum AUROC Score for an OOD Dataset


## Data Augmentation

Now we will see if models trained using data augmentation methods for robustness help in OOD detection. 

We will load a CIFAR-10 model that used PixMix data augmentation during training, and see how it fares compared to the default model that did not use data augmentation.

In [46]:
!gdown 1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68 # Downloading PixMix model
!gdown 1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa # Downlaoding wideresnet class

Downloading...
From: https://drive.google.com/uc?id=1mjIfbb3mfXXvAZ1sBnjotFr5yYFmLi68
To: /content/checkpoint.pth.tar
100% 71.8M/71.8M [00:01<00:00, 36.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1skZT6yplO-Sv4M8Ksgzx14cTY3H-HgOa
To: /content/wideresnet_with_pen.py
100% 4.03k/4.03k [00:00<00:00, 7.51MB/s]


In [47]:
from wideresnet_with_pen import WideResNet as WideResNet2

In [48]:
# Loading the PixMix model

pixmix_net = WideResNet2(depth=40, num_classes=10, widen_factor=4, drop_rate=0.3)
pixmix_net = torch.nn.DataParallel(pixmix_net)

if torch.cuda.is_available():
    checkpoint = torch.load('checkpoint.pth.tar')
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()
    net.cuda()
else:
    checkpoint = torch.load('checkpoint.pth.tar', map_location=torch.device('cpu'))
    pixmix_net.load_state_dict(checkpoint['state_dict'])
    net.eval()

## PixMix Model Testing

Let us now test the PixMix model with the same anomaly score calculators we coded before.

In [49]:
print("======= Max Logit AUROC Scores =======")
print_all_results(max_logit_anomaly_score, "Max Logit", pixmix_net, model_name = "pixmix_trained_model")

======= Max Logit AUROC Scores =======
Rademacher Noise Detection
AUROC: 			94.40%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			92.67%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			87.10%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [50]:
print("======= Max Softmax Probability AUROC Scores =======")
print_all_results(max_softmax_anomaly_score, "Max Softmax Probability", pixmix_net, model_name = "pixmix_trained_model")

======= Max Softmax Probability AUROC Scores =======
Rademacher Noise Detection
AUROC: 			92.64%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			91.31%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			87.95%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [51]:
print("======= Cross Entropy AUROC Scores =======")
print_all_results(cross_entropy_anomaly_score, "Cross Entropy", pixmix_net, model_name = "pixmix_trained_model")

======= Cross Entropy AUROC Scores =======


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Rademacher Noise Detection
AUROC: 			92.67%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


AUROC: 			91.15%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


AUROC: 			87.94%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [52]:
print("======= ViM_anomaly_score_calculator =======")
w, b = pixmix_net.module.fc.weight.cpu().detach().numpy(), pixmix_net.module.fc.bias.cpu().detach().numpy()
u = -np.matmul(pinv(w), b)
principal_space, alpha = compute_ViM_principal_space_and_alpha(cifar10_loader, pixmix_net)
print_all_results(ViM_anomaly_score_calculator, "ViM", pixmix_net, model_name = "pixmix_trained_model", use_penultimate = True)

======= ViM_anomaly_score_calculator =======
x.shape (200, 256)
Rademacher Noise Detection
AUROC: 			94.78%

SVHN Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			94.84%

CIFAR-100 Detection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


AUROC: 			92.55%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


## Compare No-data-augmentation Model vs PixMix Model

Let us now compare how the default model compared to the PixMix model by running the following cell. 

You should see that the PixMix model successfully helps us in OOD detection, and has a higher AUROC score.

In [53]:
compare_all_results()

            default_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    69.77%    |    91.43%    |    86.99%    |    82.73%   
              Max Softmax |    80.10%    |    92.32%    |   *88.72%    |    87.05%   
            Cross Entropy |    80.10%    |    91.99%    |    88.40%    |    86.83%   
                      ViM |   *99.94%    |   *95.20%    |    82.82%    |   *92.65%   

     pixmix_trained_model |  Rademacher  |     SVHN     |  CIFAR-100   |   Average   
                Max Logit |    94.40%    |    92.67%    |    87.10%    |    91.39%   
  Max Softmax Probability |    92.64%    |    91.31%    |    87.95%    |    90.63%   
            Cross Entropy |    92.67%    |    91.15%    |    87.94%    |    90.59%   
                      ViM |   *94.78%    |   *94.84%    |   *92.55%    |   *94.06%   


* highlights the maximum AUROC Score for an OOD Dataset
